In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment',  None)

In [14]:
r_food_d = pd.read_csv('output/processed_en.csv')
r_food_d.head()

,SPEAKER,SENTENCE,개체명,지식베이스,intent_cat
0,고객,Is that there jjamppong nest?,둥지 짬뽕,jjamppong|상호,매장_상품정보
1,고객,Is it a gimbap specialty restaurant?,"김밥, 전문집",gimbap|메뉴,매장_상품정보
2,고객,Is Sachensong?,사천성,Sachensong|상호,매장_상품정보
3,고객,"Is Luwan, right?",루완,Luwan|상호,매장_상품정보
4,고객,Are you liking Wolleung soba?,월령소바,Wolleung soba|상호,매장_상품정보


In [15]:
from transformers import BertTokenizer, ElectraTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')

In [16]:
r_food_d['TOKENIZED_SENTENCE'] = r_food_d['SENTENCE'].apply(lambda x: tokenizer.tokenize(x))
r_food_d['ENCODED_SENTENCE'] = r_food_d['SENTENCE'].apply(lambda x: tokenizer.encode(x))
r_food_d.head()

,SPEAKER,SENTENCE,개체명,지식베이스,intent_cat,TOKENIZED_SENTENCE,ENCODED_SENTENCE
0,고객,Is that there jjamppong nest?,둥지 짬뽕,jjamppong|상호,매장_상품정보,"[is, that, there, jj, ##amp, ##pon, ##g, nest, ?]","[101, 2003, 2008, 2045, 29017, 16613, 26029, 2..."
1,고객,Is it a gimbap specialty restaurant?,"김밥, 전문집",gimbap|메뉴,매장_상품정보,"[is, it, a, gi, ##mba, ##p, specialty, restaur...","[101, 2003, 2009, 1037, 21025, 11201, 2361, 12..."
2,고객,Is Sachensong?,사천성,Sachensong|상호,매장_상품정보,"[is, sac, ##hen, ##son, ##g, ?]","[101, 2003, 17266, 10222, 3385, 2290, 1029, 102]"
3,고객,"Is Luwan, right?",루완,Luwan|상호,매장_상품정보,"[is, lu, ##wan, ,, right, ?]","[101, 2003, 11320, 7447, 1010, 2157, 1029, 102]"
4,고객,Are you liking Wolleung soba?,월령소바,Wolleung soba|상호,매장_상품정보,"[are, you, liking, wo, ##lle, ##ung, sob, ##a, ?]","[101, 2024, 2017, 16663, 24185, 6216, 5575, 17..."


In [17]:
entity_slot = []
slot_vocab = set()

for vals in r_food_d.values:

    # raw_entity = r_food_d.iloc[i]['지식베이스']
    raw_entity = vals[3]
    encoded_sentence = vals[-1][1:-1]

    if type(raw_entity)==float: # 없는 경우
    
        entity_slot.append('')
        continue

    split_entity = raw_entity.split(', ')

    raw_entity_label = []
    raw_entity_name = []

    if len(split_entity)>1: # 여러개 ENTITY
 
        for k in split_entity:      

            entity_name = k.split('|')[0]
            entity_label = k.split('|')[1]

            raw_entity_name.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(entity_name)))
            raw_entity_label.append(entity_label)
   
    else:

        split_entity = raw_entity.split('|')

        for idx, NAME_or_LABEL in enumerate(split_entity):
            
            # [이름, 라벨]
            if idx % 2 == 0: # ENTITY_NAME

                entity_name = NAME_or_LABEL
                raw_entity_name.append(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(entity_name)))

            else: # ENTITY_LABEL

                entity_label = NAME_or_LABEL
                raw_entity_label.append(NAME_or_LABEL) 

    # sentence_token = ', '.join([str(ENCODED_SENT) for ENCODED_SENT in encoded_sentence])
   
    entity_replace_loc = []
    entity_replace_label = []
    
    for n,l in zip(raw_entity_name, raw_entity_label):

        token_num_list = [str(token_num) for token_num in n]
        bi_list = ['B-' + l if idx == 0 else 'I-'+ l for idx in range(len(token_num_list))]
        slot_vocab.add('B-' + l)
        slot_vocab.add('I-' + l)

        joined_t = ', '.join(token_num_list)
        joined_bi_list = ', '.join(bi_list)

        entity_replace_loc.append(joined_t)
        entity_replace_label.append(joined_bi_list)

    loc_label_dic = {k:v for k,v in zip(entity_replace_loc, entity_replace_label)}

    entity = ['O']* len(encoded_sentence)

    for idx, t in enumerate(entity_replace_loc):

        a = 0
        b = 0

        while a < len(encoded_sentence):

            tmp_loc = t.split(', ')
            tmp_loc = [int(t) for t in tmp_loc]

            if encoded_sentence[a] == tmp_loc[0]:

                if encoded_sentence[a:a+len(tmp_loc)] == tmp_loc:
                    
                    entity[a:a+len(tmp_loc)] = loc_label_dic[str(tmp_loc).replace('[','').replace(']','')].split(', ')

                    a += len(tmp_loc)
                    
                else:

                    a += 1

            else:

                a += 1

            
    entity_slot.append(entity)
    
slot_vocab = sorted(list(slot_vocab), key=lambda x: (x[2:], x[:2]))

r_food_d['ENTITY_SLOT'] = entity_slot

In [18]:
r_food_d.head()

,SPEAKER,SENTENCE,개체명,지식베이스,intent_cat,TOKENIZED_SENTENCE,ENCODED_SENTENCE,ENTITY_SLOT
0,고객,Is that there jjamppong nest?,둥지 짬뽕,jjamppong|상호,매장_상품정보,"[is, that, there, jj, ##amp, ##pon, ##g, nest, ?]","[101, 2003, 2008, 2045, 29017, 16613, 26029, 2...","[O, O, O, B-상호, I-상호, I-상호, I-상호, O, O]"
1,고객,Is it a gimbap specialty restaurant?,"김밥, 전문집",gimbap|메뉴,매장_상품정보,"[is, it, a, gi, ##mba, ##p, specialty, restaur...","[101, 2003, 2009, 1037, 21025, 11201, 2361, 12...","[O, O, O, B-메뉴, I-메뉴, I-메뉴, O, O, O]"
2,고객,Is Sachensong?,사천성,Sachensong|상호,매장_상품정보,"[is, sac, ##hen, ##son, ##g, ?]","[101, 2003, 17266, 10222, 3385, 2290, 1029, 102]","[O, B-상호, I-상호, I-상호, I-상호, O]"
3,고객,"Is Luwan, right?",루완,Luwan|상호,매장_상품정보,"[is, lu, ##wan, ,, right, ?]","[101, 2003, 11320, 7447, 1010, 2157, 1029, 102]","[O, B-상호, I-상호, O, O, O]"
4,고객,Are you liking Wolleung soba?,월령소바,Wolleung soba|상호,매장_상품정보,"[are, you, liking, wo, ##lle, ##ung, sob, ##a, ?]","[101, 2024, 2017, 16663, 24185, 6216, 5575, 17...","[O, O, O, B-상호, I-상호, I-상호, I-상호, I-상호, O]"
